In [2]:
import param as pm

In [26]:
class TECQFSME(pm.Parameterized):
    boost_factor = pm.Number(1, bounds=(0,4), step=0.1, doc="Multiplicative factor to apply to boosting coefficient.")
    dataset  = pm.DataFrame(columns={'amountUSD', 'projectId'}, precedence=-1, doc="Dataset of donations. Must contain amountUSD and projectId columns.")
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000, doc="Matching pool amount.")
    total_donations = pm.Number(0, constant=True, doc="Summation of amountUSD from donations dataset.")
    total_funding_pool = pm.Number(0, constant=True, doc="Summation of matching_pool and total_donations.")
    allocations  = pm.DataFrame(precedence=-1, doc="Percentages allocation table.")
    results  = pm.DataFrame(precedence=-1, doc="Matched and unmatched funding amounts. Allocation percentages times funding amounts.")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.dataset = self.qf(self.dataset)
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        """
        Takes a specefied column as the donations column. Applies the QF algorithm to produce a new specefied column and intermediate calculation columns.
        """
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @staticmethod
    def signal_boost_v1(df, boost_factor, boost_column='amountUSD', new_column_name='amount_boosted'):
        """
        Given a dataset and a specefied column, applies the flag boost algorithm.
        Requires that the dataset contain 'tec_token_flag' and 'tea_flag'.
        """
        df['coefficient'] = 1 + boost_factor * (df['tec_tokens_flag'].astype(int) | df['tea_flag'].astype(int))
        df[new_column_name] = df[boost_column] * df['coefficient']
        return df

    
    @pm.depends('boost_factor', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.total_donations = self.dataset['amountUSD'].sum()
            self.total_funding_pool = self.matching_pool + self.total_donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Generate and apply the signal boosting coefficient
            self.dataset = self.signal_boost_v1(self.dataset, self.boost_factor, boost_column='amountUSD', new_column_name='amount_boosted')

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Create an allocations table that contains allocation percentages grouped and summed by project. 
            allocation_columns = ['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])
            self.allocations = self.dataset[allocation_columns].groupby('projectId').sum()

            # Generate the unmatched results table by multiplying allocation percentages by total donations
            unmatched_results = self.total_donations * self.allocations
            
            # Generate the matched results table by multiplying allocation percentages by total donations plus matching pool
            matched_results = self.total_funding_pool * self.allocations
            
            # Merge matched and unmatched results
            self.results = unmatched_results.merge(matched_results, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            
            # Sort results by funding amount
            self.results = self.results.sort_values('quadratic_amount_allocation_matched', ascending=False)

            # Save the boosting percentage stat
            self.results['Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'] + 1)


In [27]:
import numpy as np

In [28]:
np.ones(100)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [62]:
w = 100

In [63]:
import pandas as pd

In [64]:
df = pd.DataFrame([list(np.ones(100)) + [w], ['a']*100+['b']]).T
df.columns = ['amountUSD', 'projectId']
df['amountUSD'] = df['amountUSD'].astype(float)
df['tec_tokens_flag'] = 1
df['tea_flag'] = 0
df

,amountUSD,projectId,tec_tokens_flag,tea_flag
0,1.0,a,1,0
1,1.0,a,1,0
2,1.0,a,1,0
3,1.0,a,1,0
4,1.0,a,1,0
...,...,...,...,...
96,1.0,a,1,0
97,1.0,a,1,0
98,1.0,a,1,0
99,1.0,a,1,0


In [65]:
df.dtypes

amountUSD          float64
projectId           object
tec_tokens_flag      int64
tea_flag             int64
dtype: object

In [66]:
np.sqrt(df['amountUSD'])

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
96      1.0
97      1.0
98      1.0
99      1.0
100    10.0
Name: amountUSD, Length: 101, dtype: float64

In [67]:
t = TECQFSME(dataset=df)

In [68]:
import panel as pn
pn.extension()

In [69]:
pn.Row(t)

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=200.0)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=25200.0)

In [70]:
t.dataset

,amountUSD,projectId,tec_tokens_flag,tea_flag,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount,coefficient,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
1,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
2,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
3,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
4,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
97,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
98,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600
99,1.0,a,1,0,0.005,0.009999,1.999800,2,2.0,0.005,0.009999,3.999600


In [71]:
t.results.T#[['quadratic_amount_allocation_unmatched']]

projectId,a,b
amountUSD_allocation_unmatched,100.000000,100.000000
quadratic_amount_allocation_unmatched,199.980002,0.019998
amount_boosted_allocation_unmatched,100.000000,100.000000
quadratic_amount_boosted_allocation_unmatched,199.980002,0.019998
amountUSD_allocation_matched,12600.000000,12600.000000
quadratic_amount_allocation_matched,25197.480252,2.519748
amount_boosted_allocation_matched,12600.000000,12600.000000
quadratic_amount_boosted_allocation_matched,25197.480252,2.519748
Percentage Boost,100.000000,100.000000


* Number of donors
* Describe Donations (Mean, Median, Mode)
* Expand Dataset across tegr1
* Show Distributions Quadratically